In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-07-17"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
7388,2024-07-17,Eua Nba,20:00,Orlando Magic,Memphis Grizzlies,2.16,1.68,185.5,1.90,1.88,1.5,2.00,1.51,2Fimytn4,0.462963,0.595238,0.526316,0.531915,0.058201,193.826,58.229378,0.300421,1.8,1.643168,0.912871,124.67,1.946,0.562032,0.288814,26.0,350.272,240.848689,0.687605,1.2,1.643168,1.369306,162.00,3.148,1.984608,0.630435,10.0,91,108,1.37,1.50,186.114,493.690,0.176777,0.007483,0.197426,3.37,0.674,1.721068,0.670781,0.8,0.129219,4.73,0.946,0.718816,0.262394,0.4,0.137606
7389,2024-07-17,Eua Nba,16:00,Dallas Mavericks,Miami Heat,2.59,1.49,183.5,1.89,1.89,0.0,0.00,0.00,dxVLKK1N,0.386100,0.671141,0.529101,0.529101,0.057241,243.626,50.258221,0.206293,0.6,1.341641,2.236068,225.28,2.568,0.733328,0.285564,-8.0,355.252,195.770653,0.551075,1.2,1.643168,1.369306,137.70,3.980,2.531531,0.636063,-34.0,88,102,2.56,1.35,216.908,309.514,0.381283,0.000000,NaN,0.18,0.036,44.166667,0.586719,0.6,0.013281,-1.60,-0.320,-1.531250,0.506517,0.6,0.093483
7390,2024-07-17,Eua Nba,16:30,New York Knicks,Sacramento Kings,2.35,1.58,173.5,1.89,1.89,2.5,1.99,1.49,GMBhFbPp,0.425532,0.632911,0.529101,0.529101,0.058443,277.420,63.747280,0.229786,0.0,0.000000,NaN,182.75,2.912,0.499320,0.171470,-61.0,191.738,53.354963,0.278270,1.8,1.643168,0.912871,200.90,1.960,0.565597,0.288570,48.0,85,82,2.15,2.45,206.992,230.608,0.277085,0.000000,0.203192,-0.37,-0.074,-18.243243,0.677608,0.7,0.022392,-3.62,-0.724,-0.801105,0.541411,0.4,-0.141411
7391,2024-07-17,Eua Nba,18:00,Utah Jazz,Toronto Raptors,2.13,1.70,172.5,1.89,1.89,1.5,1.97,1.54,W4Sh9Qcm,0.469484,0.588235,0.529101,0.529101,0.057719,307.612,207.255809,0.673757,1.8,1.643168,0.912871,107.80,2.856,1.680247,0.588322,-14.0,538.028,521.387427,0.969071,1.2,1.643168,1.369306,139.44,5.324,4.976307,0.934693,-13.0,70,84,1.54,1.66,358.100,791.964,0.158776,0.000000,0.173251,-2.84,-0.568,-1.989437,0.384522,0.2,-0.184522,1.08,0.216,3.240741,0.255660,0.1,-0.155660
7392,2024-07-17,Eua Nba,18:30,Charlotte Hornets,Boston Celtics,2.16,1.68,177.5,1.86,1.88,1.5,1.98,1.57,t03vw2Hi,0.462963,0.595238,0.537634,0.531915,0.058201,406.930,306.869385,0.754109,2.4,1.341641,0.559017,113.60,3.722,2.407814,0.646914,-4.0,180.432,50.163430,0.278018,1.8,1.643168,0.912871,127.60,1.800,0.366742,0.203746,-4.0,80,88,1.42,1.45,540.812,163.796,0.176777,0.007563,0.163332,-0.06,-0.012,-96.666667,0.233876,0.3,0.066124,1.35,0.270,2.518519,0.686603,0.8,0.113397
7393,2024-07-17,Eua Nba,21:00,Oklahoma City Thunder,Phoenix Suns,2.10,1.71,172.5,1.90,1.88,1.5,1.92,1.56,AueeZJGG,0.476190,0.584795,0.526316,0.531915,0.060986,202.868,62.349902,0.307342,0.6,1.341641,2.236068,230.68,2.274,0.599149,0.263478,-63.0,231.688,40.102029,0.173086,0.0,0.000000,NaN,262.26,2.488,0.807663,0.324624,-56.0,73,94,3.16,2.79,156.270,222.072,0.144762,0.007483,0.146298,-1.21,-0.242,-4.545455,0.756645,0.7,-0.056645,-1.35,-0.270,-2.629630,0.519719,0.5,-0.019719
7394,2024-07-17,Europa Campeonato Europeu Sub,14:30,Ucrânia U20,Kosovo U20,1.03,9.65,147.5,1.84,1.86,0.0,0.00,0.00,SKZkzUGF,0.970874,0.103627,0.543478,0.537634,0.074501,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,81,61,2.47,5.24,0.000,0.000,1.141435,0.007644,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
7388,20:00,Eua Nba,Orlando Magic,Memphis Grizzlies,2.16,Back Home
7401,22:30,Eua Nba,Atlanta Hawks,Los Angeles Lakers,1.65,Back Home
7407,20:30,Eua Wnba,Dallas Wings F,Indiana Fever F,2.64,Back Home
